# ДЗ 3. Коллаборативная фильтрация

# ЗАДАЧА

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере iFood. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влязает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **2 новых товара** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)
- **1 дорогой товар, > 7 долларов** (Чтобы увеличить средний чек)
- **Все товары из разных категорий**  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [134]:
path = 'C:/Users/user/Documents/mydocs/rs_geek_brains/data/'
data = pd.read_csv(path + 'retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

#последние 3 недели для теста 
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [135]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [136]:
item_features = pd.read_csv(path + 'product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


## Подготовка данных

In [137]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [138]:
#Определим нерелевантные категории для рекомендаций продуктов
notintresting = ['ELECT &PLUMBING', 'COUP/STR & MFG', 'RX', 'PROD-WHS SALES', 'AUTOMOTIVE', 'CNTRL/STORE SUP', 'HOUSEWARES', 'GM MERCH EXP', 'KIOSK-GAS', 'MISC SALES TRAN', 'MISC. TRANS.', 'TOYS', 'CHARITABLE CONT', 'PHARMACY SUPPLY', 'POSTAL CENTER', 'PHOTO', 'VIDEO', 'GARDEN CENTER']
notintresting 

['ELECT &PLUMBING',
 'COUP/STR & MFG',
 'RX',
 'PROD-WHS SALES',
 'AUTOMOTIVE',
 'CNTRL/STORE SUP',
 'HOUSEWARES',
 'GM MERCH EXP',
 'KIOSK-GAS',
 'MISC SALES TRAN',
 'MISC. TRANS.',
 'TOYS',
 'CHARITABLE CONT',
 'PHARMACY SUPPLY',
 'POSTAL CENTER',
 'PHOTO',
 'VIDEO',
 'GARDEN CENTER']

In [139]:
# определим цену 
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [140]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.82
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.99
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,1.21
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50


In [141]:
def prefilter_items(data, item_features, notintresting  = list):
    
    #data = data_train.copy()
    #для тех, кто не в топ 5000 по продажам, поставим фиктивный номер item_id 999999,
    #чтобы не терять чеки по мелким покупкам у пользователей, которые их покупали, но и не рекомендовать мелкие покупки 
    
    '''
    popularity_sum = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity_sum.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity_sum.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    data.loc[~data['item_id'].isin(top_5000), 'item_id'] = 999999
    '''
    # Уберем самые популярные товары типа молоко и хлеб  (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 3 месяца
    notsold = data['item_id'][(data['week_no'] <=3*4 ) & (data['sales_value'] == 0)]
    data = data[~data['item_id'].isin( notsold)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    notint =  item_features['item_id'].loc[item_features['department'].isin(notintresting)]
    data = data[~data['item_id'].isin(notint)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    cheap = data['item_id'].loc[data['price'] < 1]
    data = data[~data['item_id'].isin(cheap)]
    
    # Уберем слишком дорогие товарыs
    expensive = data['item_id'].loc[data['price'] > data.price.quantile(0.97)]
    data = data[~data['item_id'].isin(expensive)]
    
    return data

In [142]:
data_train = prefilter_items(data_train, item_features, notintresting)

In [143]:
data_train.sample(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
1291577,1975,32759907903,389,915545,1,3.19,448,0.00,1806,56,0.0,0.0,3.19
1993827,1663,40279178724,565,15716085,1,1.00,406,-0.39,2158,81,0.0,0.0,1.00
1567146,1378,33769878347,458,9855865,2,6.40,381,0.00,2005,66,0.0,0.0,3.20


In [144]:
#Когда мы убрали товары, которые не продавались последние три мес, пропал наш фиктивный item 999999 
data_train.loc[data_train['item_id']==999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price


In [145]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890984 entries, 7 to 2282324
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            890984 non-null  int64  
 1   basket_id          890984 non-null  int64  
 2   day                890984 non-null  int64  
 3   item_id            890984 non-null  int64  
 4   quantity           890984 non-null  int64  
 5   sales_value        890984 non-null  float64
 6   store_id           890984 non-null  int64  
 7   retail_disc        890984 non-null  float64
 8   trans_time         890984 non-null  int64  
 9   week_no            890984 non-null  int64  
 10  coupon_disc        890984 non-null  float64
 11  coupon_match_disc  890984 non-null  float64
 12  price              890984 non-null  float64
dtypes: float64(5), int64(8)
memory usage: 95.2 MB


## Подоготовим матрицу user_item_matrix

In [146]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26093,26190,26426,26601,26636,26691,26738,26889,26941,...,17291665,17319084,17320734,17320911,17328261,17328742,17329749,17330255,17383227,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
sparse_user_item.get_shape

<bound method spmatrix.get_shape of <2498x64125 sparse matrix of type '<class 'numpy.float64'>'
	with 584716 stored elements in Compressed Sparse Row format>>

In [148]:
user_item_matrix.max().max()

129.0

In [149]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## Модель ALS

In [150]:
%%time

model = AlternatingLeastSquares(factors=100,  # Нужно подбирать
                                regularization=0.001,
                                iterations=25,
                                calculate_training_loss=True, 
                                num_threads=2)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 8.77 s


In [151]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_items=None, 
                        recalculate_user=False)

In [152]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], #берем id из словаря 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    #filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [153]:
recs

[(43926, 0.26751566),
 (8504, 0.26586643),
 (10455, 0.25122714),
 (43148, 0.2456135),
 (18863, 0.24424648)]

In [154]:
def precision_at_k(recommended_list, bought_list, k = 5 ):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

In [155]:
results_users = [x for x in data_train.user_id.unique()]

In [156]:
result = result.loc[ result['user_id'].isin(results_users)]

In [157]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual'], k=5), axis=1).mean()

Wall time: 6min 37s


0.09465948064674132

0.09593336599705976

In [158]:
result.head()

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 7025114, 9527290, 865456, 856942]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9526410, 1101173, 9527494, 998206, 907631]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[821083, 9527290, 9526410, 1041796, 9911484]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5591154, 873203, 963502, 871440, 12384365]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1075214, 8065410, 1041796, 9527290, 872137]"


## Попробуем TF-IDF взвешивание

In [159]:
user_item_matrix_tf = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [160]:
%%time

model_tf = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4)

model_tf.fit(csr_matrix(user_item_matrix_tf).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.59 s


In [161]:
result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_tf, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.08642822146006834

0.08770210681038679

In [162]:
result.head()

,user_id,actual,als,als_tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 7025114, 9527290, 865456, 856942]","[9526410, 9527290, 940947, 944534, 5569374]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9526410, 1101173, 9527494, 998206, 907631]","[1101173, 998206, 972931, 9527494, 883932]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[821083, 9527290, 9526410, 1041796, 9911484]","[835098, 10149640, 821083, 825749, 996087]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5591154, 873203, 963502, 871440, 12384365]","[916993, 963502, 835285, 1091499, 5591154]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1075214, 8065410, 1041796, 9527290, 872137]","[823704, 872137, 845622, 8065410, 9527290]"


### 3. Попробуем BM25 взвешивание

In [163]:
user_item_matrix_bm = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [164]:
%%time

model_bm = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model_bm.fit(csr_matrix(user_item_matrix_bm).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.72 s


In [165]:
result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_bm, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.13130818226359534

0.12974032337089575

## Вывод

При текущих дополнениях по бизнес-правилам на этапе предобработки точность значительно снизилась. Даже при увеличении количества итераций, она остается на прежнем уровне. Лучший результат показала модель с BM25 взвешиванием. 

----